## Pisos turisticos Euskadi COMPLETO
Fuente: https://www.ine.es/jaxiT3/Datos.htm?t=39364

In [32]:
import numpy as np
import pandas as pd

Fuente: https://www.ine.es/jaxiT3/Datos.htm?t=39364

In [47]:
ine_pisos_tur = pd.read_csv('./data/viviendas_turisticas_INE.csv', encoding='latin-1')


In [34]:
import pandas as pd

df = pd.read_csv("./data/viviendas_turisticas_INE.csv", sep=';', encoding='latin1')

df = df[df["Viviendas y plazas"] == "Viviendas turísticas"]
df["Total"] = df["Total"].astype(str).str.replace('.', '', regex=False)
df["Año"] = df["Periodo"].str[:4]
df["Viviendas"] = pd.to_numeric(df["Total"], errors='coerce')

prov_df = df[df["Provincias"].notna()].copy()
prov_df["Entidad"] = prov_df["Provincias"].str.extract(r'\d{2}\s+(.*)')
prov_df["Entidad"] = prov_df["Entidad"].replace({
    "Araba/Álava": "Araba",
    "Bizkaia": "Bizkaia",
    "Gipuzkoa": "Gipuzkoa"
})
prov_max = prov_df.groupby(["Entidad", "Año"])["Viviendas"].max().reset_index()


cae_df = df[(df["Provincias"].isna()) & (df["Comunidades y Ciudades Autónomas"] == "16 País Vasco")]
cae_max = cae_df.groupby("Año")["Viviendas"].max().reset_index()
cae_max["Entidad"] = "CAE"

final_df = pd.concat([prov_max, cae_max], ignore_index=True)

df_pivot = final_df.pivot(index="Entidad", columns="Año", values="Viviendas")

df_pivot = df_pivot[sorted(df_pivot.columns, reverse=True)]

df_final = df_pivot.reset_index()
df_final.columns.name = None
df_final = df_final.rename(columns={"Entidad": "Territorio"})

df_final.to_csv("./data/viviendas_turisticas.csv", sep=';', index=False)



## Precio alquiler Euskadi - Territorios históricos COMPLETO
Fuente: https://www.eustat.eus/elementos/ele0017700/renta-mensual-media-de-los-contratos-de-alquiler-libre-de-vivienda-habitual-colectiva-contrato-de-vivienda-habitual-por-territorio-historico-y-tamano-del-municipio-segun-trimestre-de-inicio-del-contrato-eurosmes/tbl0017789_c.html

In [35]:
import pandas as pd

archivo = "./data/precio_alquiler.csv"

df = pd.read_csv(archivo, sep=';', skiprows=5, header=None, encoding='latin1')

territorios = ['C.A. de Euskadi', '   Araba/Álava', '   Bizkaia', '   Gipuzkoa']
df = df[df[0].isin(territorios)].reset_index(drop=True)

df[0] = df[0].str.strip().replace({
    'C.A. de Euskadi': 'CAE',
    'Araba/Álava': 'Araba',
    'Bizkaia': 'Bizkaia',
    'Gipuzkoa': 'Gipuzkoa'
})

df = df.dropna(axis=1, how='all')

data_only = df.iloc[:, 1:].applymap(lambda x: str(x).replace(',', '.')).astype(float)

años = list(range(2016, 2025))
n_trimestres = 4

columnas_anuales = {}
for i, año in enumerate(años):
    inicio = i * n_trimestres
    fin = inicio + n_trimestres
    columnas_anuales[año] = data_only.iloc[:, inicio:fin].mean(axis=1)

df_resultado = pd.DataFrame(columnas_anuales)
df_resultado.insert(0, 'Territorio', df[0])


df_resultado.to_csv('./data/precio_alquiler_.csv')

df_resultado


C:\Users\ortas\AppData\Local\Temp\ipykernel_2532\2649944901.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_only = df.iloc[:, 1:].applymap(lambda x: str(x).replace(',', '.')).astype(float)


,Territorio,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,CAE,628.611352,638.736288,667.306500,688.964002,702.334793,709.413165,735.088313,776.313490,795.721470
1,Araba,544.793245,559.165458,592.394645,621.516687,634.338480,644.832238,678.004437,718.343090,723.885465
2,Bizkaia,644.169768,655.916545,690.234163,713.666588,729.841342,737.460952,762.392385,809.144362,828.982632
3,Gipuzkoa,642.550235,650.212623,671.556517,690.769225,700.095973,706.987067,731.464312,770.008853,792.404865


In [36]:
import pandas as pd

df = pd.read_csv("./data/precio_alquiler_.csv", sep=',', encoding="latin1")
df = df.drop(columns=df.columns[0])

df.to_csv('./data/precio_alquiler_final.csv', index=False)




## Precio compraventa libre - COMPLETO
Fuente: https://www.eustat.eus/elementos/ele0013600/precio-medio-total-de-las-compraventas-de-vivienda-libre-de-la-ca-de-euskadi-por-trimestre-de-inscripcion-en-el-registro-segun-territorio-historico-y-tipo-de-vivienda-/tbl0013664_c.html

In [42]:
import pandas as pd

def limpiar_csv_compraventa(ruta_csv, ruta_salida=None):
    
    df = pd.read_csv(ruta_csv, sep=';', skiprows=6, encoding='latin1', header=None)

    df.dropna(axis=1, how='all', inplace=True)

    columnas = ['Trimestre', 
                'Euskadi_Total', 'Euskadi_Nueva', 'Euskadi_Usada',
                'Álava_Total', 'Álava_Nueva', 'Álava_Usada',
                'Bizkaia_Total', 'Bizkaia_Nueva', 'Bizkaia_Usada',
                'Gipuzkoa_Total', 'Gipuzkoa_Nueva', 'Gipuzkoa_Usada']
    df.columns = columnas

    for col in columnas[1:]:
        df[col] = pd.to_numeric(df[col], errors='coerce')

        df.to_csv(ruta_salida, index=False)

    return df

# df_limpio = limpiar_csv_compraventa("precio_compraventa_libre.csv", "compraventa_limpio.csv")


In [43]:
limpiar_csv_compraventa('./data/precio.csv','./data/precio_cv_limpio.csv')

,Trimestre,Euskadi_Total,Euskadi_Nueva,Euskadi_Usada,Álava_Total,Álava_Nueva,Álava_Usada,Bizkaia_Total,Bizkaia_Nueva,Bizkaia_Usada,Gipuzkoa_Total,Gipuzkoa_Nueva,Gipuzkoa_Usada
0,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,I,267114.0,297774.0,260271.0,224678.0,230595.0,222308.0,247311.0,279833.0,239124.0,315334.0,408446.0,303217.0
2,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IV,266692.0,285388.0,262782.0,236454.0,230058.0,238988.0,249356.0,262106.0,246462.0,303779.0,390071.0,292504.0
4,III,254463.0,285509.0,248547.0,226078.0,242534.0,218616.0,237741.0,281675.0,228817.0,292993.0,371597.0,286166.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,Tabla elaborada con datos del Colegio de Regis...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:

precio_cv_limpio = pd.read_csv('./data/precio_cv_limpio.csv')
precio_cv_limpio


,Trimestre,Euskadi_Total,Euskadi_Nueva,Euskadi_Usada,Álava_Total,Álava_Nueva,Álava_Usada,Bizkaia_Total,Bizkaia_Nueva,Bizkaia_Usada,Gipuzkoa_Total,Gipuzkoa_Nueva,Gipuzkoa_Usada
0,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,I,267114.0,297774.0,260271.0,224678.0,230595.0,222308.0,247311.0,279833.0,239124.0,315334.0,408446.0,303217.0
2,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IV,266692.0,285388.0,262782.0,236454.0,230058.0,238988.0,249356.0,262106.0,246462.0,303779.0,390071.0,292504.0
4,III,254463.0,285509.0,248547.0,226078.0,242534.0,218616.0,237741.0,281675.0,228817.0,292993.0,371597.0,286166.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,Tabla elaborada con datos del Colegio de Regis...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# Función para cambiar los NaN de los años por la media de los trimestres y borrar los datos de los trimestres
import pandas as pd

def precio_año_media(df):
    df = df.copy()
    filas_a_conservar = []
    
    i = 0
    while i < len(df) - 4:
        fila = df.iloc[i]
        if str(fila['Trimestre']).isdigit() and pd.isna(fila[1:]).all():
            # Fila con año y NaNs: calcular media de los 4 trimestres siguientes
            medias = df.iloc[i+1:i+5].mean(numeric_only=True)
            fila_actualizada = fila.copy()
            fila_actualizada[1:] = medias.values
            filas_a_conservar.append(fila_actualizada)
            i += 5  # Saltar año + 4 trimestres
        else:
            i += 1  

    df_anual = pd.DataFrame(filas_a_conservar).reset_index(drop=True)
    return df_anual

df_venta_anual = precio_año_media(precio_cv_limpio)

df_venta_anual.head()

precio_venta_antestrasp = df_venta_anual.to_csv('./data/precio_venta_antestrasp.csv')


In [60]:
import pandas as pd

# Cargar CSV (la primera columna parece un índice sin nombre, se ignora)
df = pd.read_csv('./data/precio_venta_antestrasp.csv', index_col=0)

# Filtrar columnas que quieres: 'Trimestre' + las que terminan en '_Total'
cols_totales = [col for col in df.columns if col.endswith('_Total')]
cols_seleccionadas = ['Trimestre'] + cols_totales
df_totales = df_totales.rename(columns={'Trimestre': 'Año'})
df_totales = df[cols_seleccionadas]

print(df_totales.head())




   Trimestre  Euskadi_Total  Álava_Total  Bizkaia_Total  Gipuzkoa_Total
0       2025  262756.333333    229070.00  244802.666667   304035.333333
1       2023  247215.000000    221106.75  234635.750000   276812.750000
2       2022  243582.250000    207139.25  236409.500000   271810.250000
3       2021  246830.750000    202848.75  241695.500000   271371.750000
4       2020  241276.000000    198174.50  237037.250000   265791.250000


In [66]:
import pandas as pd

df = pd.read_csv('./data/precio_venta_antestrasp.csv', index_col=0)

# Seleccionar columnas totales y la columna Trimestre
cols_totales = [col for col in df.columns if col.endswith('_Total')]
df_totales = df[['Trimestre'] + cols_totales]

# Renombrar Trimestre a Año para claridad
df_totales = df_totales.rename(columns={'Trimestre': 'Año'})

# Transponer: poner años como columnas, territorios como filas
df_transpuesto = df_totales.set_index('Año').T
# Renombrar filas (índice)
df_transpuesto = df_transpuesto.rename(index={
    'Euskadi_Total': 'CAE',
    'Álava_Total': 'Araba',
    'Bizkaia_Total': 'Bizkaia',
    'Gipuzkoa_Total': 'Gipuzkoa'
})
df_transpuesto.columns = df_transpuesto.columns.astype(int)
df_transpuesto = df_transpuesto.rename(columns={2025: 2024})
df_transpuesto.index.name = 'Territorio'


print(df_transpuesto.head())
df_transpuesto.to_csv('./data/precio_cv_final.csv')


Año                  2024       2023       2022       2021       2020  \
Territorio                                                              
CAE         262756.333333  247215.00  243582.25  246830.75  241276.00   
Araba       229070.000000  221106.75  207139.25  202848.75  198174.50   
Bizkaia     244802.666667  234635.75  236409.50  241695.50  237037.25   
Gipuzkoa    304035.333333  276812.75  271810.25  271371.75  265791.25   

Año              2019       2018       2017       2016      2015       2014  \
Territorio                                                                    
CAE         222329.25  212776.50  208043.75  208165.25  200153.5  204741.25   
Araba       182892.00  165228.00  167638.25  157015.25  167763.0  169159.75   
Bizkaia     217864.50  209359.25  202720.50  208024.25  200141.0  201503.50   
Gipuzkoa    242861.50  235729.75  230885.50  224898.75  210553.5  225429.00   

Año             2013       2012  
Territorio                       
CAE         219858

## Licencias de vivienda nueva COMPLETO
Fuente: https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-licencias-de-vivienda-nueva-concedidas-en-el-ultimo-quinquenio-x2030-habitantes/

In [77]:

import pandas as pd

archivo = './data/licencias.csv'

df = pd.read_csv(archivo, sep=';', header=None, encoding='latin1')

idx_inicio = df[df[0] == 'Entidad'].index[0]
idx_fin = df[df[0] == 'Gipuzkoa'].index[0]

df = df.loc[idx_inicio:idx_fin].reset_index(drop=True)

df = df.dropna(axis=1, how='all')

df = df.replace('"', '', regex=True)

df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

for col in df.columns[1:]:
    df[col] = df[col].str.replace(',', '.')
    df[col] = pd.to_numeric(df[col], errors='coerce')  

df = df.rename(columns={'Entidad': 'Territorio'})
df['Territorio'] = df['Territorio'].replace({'Araba/Álava': 'Araba'})

df.to_csv('./data/licencias_limpio.csv', index=False, sep=',')

df.head()


#Esto sigue siendo cada 1000 habitantes

,Territorio,2023,2022,2021,2020,2019,2018,2017,2016,2015,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,Araba,16.13,13.85,12.31,9.13,7.84,6.51,5.78,7.63,13.98,...,41.87,58.22,66.29,67.98,70.64,72.58,53.49,46.09,44.55,41.18
1,Bizkaia,9.12,9.03,9.03,9.58,9.61,9.09,8.31,7.40,8.20,...,10.24,14.72,17.28,20.98,25.40,27.76,25.76,29.55,31.34,53.24
2,CAE,10.35,9.76,9.75,9.57,9.55,8.72,7.92,7.59,9.23,...,16.63,22.52,26.44,29.50,32.78,34.77,29.67,31.00,31.97,32.06
3,Gipuzkoa,9.67,9.05,9.74,9.76,10.24,9.14,8.27,7.88,8.75,...,15.74,19.33,23.73,26.48,28.26,29.83,25.83,26.92,27.65,28.36


In [10]:
import pandas as pd

df_poblacion = pd.read_csv('./data/poblacion_total_limpio.csv')
df_licencias = pd.read_csv('./data/licencias_limpio.csv')

columnas_comunes = list(set(df_poblacion.columns[1:]) & set(df_licencias.columns[1:]))
columnas_comunes = sorted(columnas_comunes, reverse=True)

df_poblacion = df_poblacion[['Territorio'] + columnas_comunes].sort_values('Territorio').reset_index(drop=True)
df_licenciass = df_licencias[['Territorio'] + columnas_comunes].sort_values('Territorio').reset_index(drop=True)

df_totales = df_licencias.copy()
for col in columnas_comunes:
    df_totales[col] = ((df_licencias[col] / 1000) * df_poblacion[col]).round().astype('Int64')

df_totales.to_csv('./data/licencias_totales.csv', index=False)

print(df_totales.head())
df_totales.to_csv('./data/licencias_final.csv', index=False, sep=',')



  Territorio   2023   2022   2021   2020   2019   2018   2017   2016   2015  \
0      Araba   5369   4580   4065   3015   2571   2119   1870   2459   4498   
1    Bizkaia  10433  10287  10331  11008  10983  10369   9469   8428   9360   
2        CAE  22736  21340  21384  21051  20896  19014  17232  16485  20059   
3   Gipuzkoa   6961   6485   7002   7032   7344   6528   5895   5600   6212   

   ...   2012   2011   2010   2009   2008   2007   2006   2005   2004   2003  
0  ...  13476  18556  21015  21316  21883  22190  16143  13742  13098  11985  
1  ...  11818  16963  19902  24151  29068  31589  29039  33167  35091  59621  
2  ...  36280  48959  57349  63807  70403  74095  62763  65236  66978  67004  
3  ...  11106  13587  16618  18490  19600  20503  17727  18408  18838  19258  

[4 rows x 22 columns]


## Suelo urbanizable COMPLETO
Fuente: https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-urbanismo/


In [16]:
import pandas as pd

archivo = './data/suelo_urbanizable.csv'

df = pd.read_csv(archivo, sep=';', header=None, encoding='latin1')

idx_inicio = df[df[0] == 'Entidad'].index[0]
idx_fin = df[df[0] == 'Gipuzkoa'].index[0]

df = df.loc[idx_inicio:idx_fin].reset_index(drop=True)

df = df.dropna(axis=1, how='all')

df = df.replace('"', '', regex=True)

df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

for col in df.columns[1:]:
    df[col] = df[col].str.replace(',', '.')
    df[col] = pd.to_numeric(df[col], errors='coerce')  

df = df.rename(columns={'Entidad': 'Territorio'})
df['Territorio'] = df['Territorio'].replace({'Araba/Álava': 'Araba'})

df.to_csv('./data/suelo_urbanizable_final.csv', index=False, sep=',')

df.head()




,Territorio,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004
0,Araba,12.33,12.48,12.49,12.66,12.67,12.69,16.86,16.87,17.08,17.60,27.15,26.97,27.07,27.17,27.28,26.65,26.98,26.59,26.22
1,Bizkaia,11.71,11.98,12.06,12.02,11.85,12.05,13.15,13.22,13.43,13.32,21.00,21.79,21.67,21.82,21.92,22.24,22.15,21.91,23.78
2,CAE,11.70,12.10,12.14,12.17,12.12,12.21,14.16,14.09,14.50,14.59,21.99,22.38,22.48,22.58,22.43,22.61,22.62,22.42,23.26
3,Gipuzkoa,11.02,11.92,11.92,11.92,12.05,11.99,13.16,12.78,13.78,13.83,17.86,18.37,18.86,18.86,17.78,18.43,18.23,18.06,17.96


## Solicitudes de vivienda que constan en Etxebide ( x1000 habitantes) COMPLETO
Fuente: https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-vivienda/

In [14]:
import pandas as pd

archivo = './data/solicitudes_vivienda.csv'

df = pd.read_csv(archivo, sep=';', header=None, encoding='latin1')

idx_inicio = df[df[0] == 'Entidad'].index[0]
idx_fin = df[df[0] == 'Gipuzkoa'].index[0]

df = df.loc[idx_inicio:idx_fin].reset_index(drop=True)

df = df.dropna(axis=1, how='all')

df = df.replace('"', '', regex=True)

df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

for col in df.columns[1:]:
    df[col] = df[col].str.replace(',', '.')
    df[col] = pd.to_numeric(df[col], errors='coerce')  

df = df.rename(columns={'Entidad': 'Territorio'})
df['Territorio'] = df['Territorio'].replace({'Araba/Álava': 'Araba'})

df.to_csv('./data/solicitudes_vivienda_limpio.csv', index=False, sep=',')

df.head()


#Esto sigue siendo cada 1000 habitantes

,Territorio,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005
0,Araba,45.48,37.30,36.91,35.33,32.37,29.80,27.56,22.75,21.98,24.04,30.83,28.93,37.92,37.70,45.09,53.05,57.82,59.66,61.13
1,Bizkaia,69.39,52.36,55.16,54.52,49.71,45.26,43.63,37.45,37.59,36.69,42.55,40.66,40.56,39.85,41.92,38.33,37.74,37.07,35.16
2,CAE,62.66,50.71,52.55,51.59,46.32,42.47,40.32,34.10,34.10,33.74,40.56,38.50,40.70,40.76,44.03,43.33,44.08,43.64,41.13
3,Gipuzkoa,59.93,54.29,55.60,54.38,47.32,43.81,40.86,33.89,34.01,33.40,41.74,39.31,42.18,43.62,47.02,47.22,48.41,47.40,42.16


In [21]:
import pandas as pd

df_poblacion = pd.read_csv('./data/poblacion_total_limpio.csv')
df_solicitudes = pd.read_csv('./data/solicitudes_vivienda_limpio.csv')

columnas_comunes = list(set(df_poblacion.columns[1:]) & set(df_solicitudes.columns[1:]))
columnas_comunes = sorted(columnas_comunes, reverse=True)

df_poblacion = df_poblacion[['Territorio'] + columnas_comunes].sort_values('Territorio').reset_index(drop=True)
df_solicitudes = df_solicitudes[['Territorio'] + columnas_comunes].sort_values('Territorio').reset_index(drop=True)

df_totales = df_solicitudes.copy()
for col in columnas_comunes:
    df_totales[col] = ((df_solicitudes[col] / 1000) * df_poblacion[col]).round().astype('Int64')

df_totales.to_csv('./data/solicitantes_totales.csv', index=False)

print(df_totales.head())


  Territorio    2023    2022    2021    2020    2019   2018   2017   2016  \
0      Araba   15138   12335   12187   11666   10616   9700   8918   7333   
1    Bizkaia   79383   59649   63110   62646   56811  51626  49713  42650   
2        CAE  137648  110878  115253  113483  101349  92604  87729  74061   
3   Gipuzkoa   43142   38905   39970   39179   33938  31292  29125  24086   

    2015   2014   2013   2012   2011   2010   2009   2008   2007   2006   2005  
0   7073   7694   9874   9311  12086  11952  14138  16434  17677  18005  18227  
1  42907  41981  48966  46927  46742  45896  48255  43866  42946  41789  39464  
2  74106  73313  88378  83991  88483  88410  95235  93062  93934  92315  86554  
3  24147  23668  29547  27737  29648  30548  32833  32749  33273  32530  28830  


## Viviendas adjudicadas por Etxebide en el ultimo trienio por cada 100 solicitudes inscritas  
## COMPLETO
Fuente:  https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-vivienda/

In [17]:
import pandas as pd

archivo = './data/viviendas_etxebide.csv'

df = pd.read_csv(archivo, sep=';', header=None, encoding='latin1')

idx_inicio = df[df[0] == 'Entidad'].index[0]
idx_fin = df[df[0] == 'Gipuzkoa'].index[0]

df = df.loc[idx_inicio:idx_fin].reset_index(drop=True)

df = df.dropna(axis=1, how='all')

df = df.replace('"', '', regex=True)

df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

for col in df.columns[1:]:
    df[col] = df[col].str.replace(',', '.')
    df[col] = pd.to_numeric(df[col], errors='coerce')  

df = df.rename(columns={'Entidad': 'Territorio'})
df['Territorio'] = df['Territorio'].replace({'Araba/Álava': 'Araba'})

df.to_csv('./data/viviendas_etxebide_limpio.csv', index=False, sep=',')

df.head()


#Esto sigue siendo cada 1000 habitantes

,Territorio,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,Araba,2.91,1.89,0.73,0.39,0.00,1.55,1.73,5.37,3.38,3.10,2.07,8.77,13.71,14.37,7.94,7.68,7.67
1,Bizkaia,0.71,1.13,1.06,1.27,1.47,1.74,1.78,1.03,2.10,2.24,3.68,3.45,3.47,3.17,2.19,2.99,2.44
2,CAE,0.94,1.02,0.89,0.97,1.08,1.42,1.56,1.58,2.33,2.39,3.34,3.76,4.83,4.71,3.23,3.52,3.45
3,Gipuzkoa,0.67,0.59,0.66,0.66,0.78,0.84,1.13,1.38,2.40,2.42,3.24,2.34,3.22,2.62,2.34,1.99,2.34


In [19]:
import pandas as pd

df_poblacion = pd.read_csv('./data/poblacion_total_limpio.csv')
df_etxebide = pd.read_csv('./data/viviendas_etxebide_limpio.csv')

columnas_comunes = list(set(df_poblacion.columns[1:]) & set(df_etxebide.columns[1:]))
columnas_comunes = sorted(columnas_comunes, reverse=True)

df_poblacion = df_poblacion[['Territorio'] + columnas_comunes].sort_values('Territorio').reset_index(drop=True)
df_etxebide = df_etxebide[['Territorio'] + columnas_comunes].sort_values('Territorio').reset_index(drop=True)

df_totales = df_etxebide.copy()
for col in columnas_comunes:
    df_totales[col] = ((df_etxebide[col] / 100) * df_poblacion[col]).round().astype('Int64')

df_totales.to_csv('./data/viviendas_etxebide_total.csv', index=False)

print(df_totales.head())


  Territorio   2023   2022   2021   2020   2019   2018   2017   2016   2015  \
0      Araba   9686   6250   2410   1288      0   5046   5598  17309  10876   
1    Bizkaia   8123  12873  12128  14593  16800  19848  20282  11730  23970   
2        CAE  20649  22302  19519  21337  23631  30962  33943  34316  50636   
3   Gipuzkoa   4823   4228   4745   4755   5594   6000   8055   9808  17040   

    2014   2013   2012    2011    2010   2009   2008   2007  
0   9921   6630  28227   43698   45555  24897  23792  23450  
1  25630  42349  39818   39988   36509  25210  34218  27766  
2  51932  72777  82028  105006  102162  69863  75601  73519  
3  17149  22936  16511   22633   18348  16339  13802  16083  


## Poblacion total (hab.) COMPLETO
Fuente: https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-vivienda/  
Necesario para normalizar índices de tablas que son porcentuales como las 2 de arriba

In [84]:
import pandas as pd

archivo = './data/Poblacion_total.csv'

# Leer archivo original
df = pd.read_csv(archivo, sep=';', header=None, encoding='latin1')

# Localizar la tabla entre 'Entidad' y 'Gipuzkoa'
idx_inicio = df[df[0] == 'Entidad'].index[0]
idx_fin = df[df[0] == 'Gipuzkoa'].index[0]
df = df.loc[idx_inicio:idx_fin].reset_index(drop=True)

# Eliminar columnas vacías
df = df.dropna(axis=1, how='all')

# Asignar encabezados
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

# Eliminar comillas
df = df.replace('"', '', regex=True)

# Limpieza de valores: quitar puntos, cambiar coma a punto, convertir a número
for col in df.columns[1:]:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.strip().str.replace('.', '', regex=False)  # quitar miles
    df[col] = df[col].str.replace(',', '.', regex=False)             # decimal
    df[col] = pd.to_numeric(df[col], errors='coerce')                # convertir

# Renombrar
df = df.rename(columns={'Entidad': 'Territorio'})
df['Territorio'] = df['Territorio'].replace({'Araba/Álava': 'Araba'})

# Guardar CSV limpio
df.to_csv('./data/poblacion_total_limpio.csv', index=False, sep=',')

df.head()


,Territorio,2024,2023,2022,2021,2020,2019,2018,2017,2016,...,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001
0,Araba,335146.0,332851.0,330692.0,330189.0,330209.0,327967.0,325518.0,323592.0,322335.0,...,317016.0,313561.0,309786.0,305732.0,301800.0,298166.0,294012.0,291038.0,287560.0,284596.0
1,Bizkaia,1150037.0,1144019.0,1139209.0,1144123.0,1149044.0,1142853.0,1140662.0,1139426.0,1138852.0,...,1151708.0,1151125.0,1144424.0,1137948.0,1127291.0,1122410.0,1119697.0,1119849.0,1120591.0,1122180.0
2,CAE,2208007.0,2196745.0,2186517.0,2193199.0,2199711.0,2188017.0,2180449.0,2175819.0,2171886.0,...,2169038.0,2162953.0,2147754.0,2130996.0,2115383.0,2104396.0,2095014.0,2089950.0,2085058.0,2079210.0
3,Gipuzkoa,722824.0,719875.0,716616.0,718887.0,720458.0,717197.0,714269.0,712801.0,710699.0,...,700314.0,698267.0,693544.0,687316.0,686292.0,683820.0,681305.0,679063.0,676907.0,672434.0


## Población extranjera
Fuente: https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-vivienda/  
Necesario para una hipotesis

In [2]:
import pandas as pd

archivo = './data/poblacion_extranjera.csv'

df = pd.read_csv(archivo, sep=';', header=None, encoding='latin1')

idx_inicio = df[df[0] == 'Entidad'].index[0]
idx_fin = df[df[0] == 'Gipuzkoa'].index[0]

df = df.loc[idx_inicio:idx_fin].reset_index(drop=True)

df = df.dropna(axis=1, how='all')

df = df.replace('"', '', regex=True)

df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

for col in df.columns[1:]:
    df[col] = df[col].str.replace(',', '.')
    df[col] = pd.to_numeric(df[col], errors='coerce')  

df = df.rename(columns={'Entidad': 'Territorio'})
df['Territorio'] = df['Territorio'].replace({'Araba/Álava': 'Araba'})

df.to_csv('./data/poblacion_extranjera_limpio.csv', index=False, sep=',')

df.head()


#Esto sigue siendo cada 1000 habitantes

,Territorio,2024,2023,2022,2021,2020,2019,2018,2017,2016,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,Araba,11.19,10.57,9.98,9.69,9.88,9.28,8.67,8.85,8.68,...,10.04,9.39,9.03,8.30,7.22,6.02,5.15,4.30,3.46,2.71
1,Bizkaia,9.43,8.85,8.05,7.87,7.93,7.04,6.37,6.61,6.27,...,6.93,6.59,6.17,5.78,5.05,4.17,3.42,2.81,2.22,1.69
2,CAE,9.84,9.27,8.54,8.34,8.41,7.62,6.98,7.15,6.88,...,7.34,6.95,6.54,6.12,5.32,4.36,3.61,2.94,2.37,1.84
3,Gipuzkoa,9.86,9.34,8.63,8.46,8.49,7.78,7.18,7.25,7.03,...,6.78,6.42,6.02,5.70,4.94,3.93,3.24,2.55,2.14,1.73


In [6]:
import pandas as pd

df_poblacion = pd.read_csv('./data/poblacion_total_limpio.csv')
df_extranjeros = pd.read_csv('./data/poblacion_extranjera_limpio.csv')

columnas_comunes = list(set(df_poblacion.columns[1:]) & set(df_extranjeros.columns[1:]))
columnas_comunes = sorted(columnas_comunes, reverse=True)

df_poblacion = df_poblacion[['Territorio'] + columnas_comunes].sort_values('Territorio').reset_index(drop=True)
df_extranjeross = df_extranjeros[['Territorio'] + columnas_comunes].sort_values('Territorio').reset_index(drop=True)

df_totales = df_extranjeros.copy()
for col in columnas_comunes:
    df_totales[col] = ((df_extranjeros[col] / 1000) * df_poblacion[col]).round().astype('Int64')

df_totales.to_csv('./data/poblacion_extranjera_total.csv', index=False)

print(df_totales.head())


  Territorio   2024   2023   2022   2021   2020   2019   2018   2017   2016  \
0      Araba   3750   3518   3300   3200   3262   3044   2822   2864   2798   
1    Bizkaia  10845  10125   9171   9004   9112   8046   7266   7532   7141   
2        CAE  21727  20364  18673  18291  18500  16673  15220  15557  14943   
3   Gipuzkoa   7127   6724   6184   6082   6117   5580   5128   5168   4996   

   ...   2012   2011   2010   2009   2008  2007  2006  2005  2004  2003  
0  ...   3231   2993   2863   2603   2237  1841  1554  1282  1017   789  
1  ...   7998   7594   7106   6654   5779  4745  3855  3154  2486  1893  
2  ...  16013  15110  14186  13237  11426  9291  7637  6187  4965  3846  
3  ...   4784   4513   4216   3980   3426  2701  2224  1744  1458  1175  

[4 rows x 23 columns]


## Necesidad de vivienda
Fuente: https://www.eustat.eus/elementos/ele0013400/cuantificacion-de-la-necesidad-de-acceso-a-una-vivienda-en-la-ca-de-euskadi-por-situacion-de-emancipacion-territorio-historico-y-sexo-segun-ingresos-o-rentas-propias-disponibles/tbl0013463_c.html


## Mujeres electas en elecciones municipales (% sobre total personas electas).csv
Fuente: Fuente: https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-vivienda/  
Hipótesis por determinar todavía


In [4]:
import pandas as pd

archivo = './data/mujeres_electas.csv'

df = pd.read_csv(archivo, sep=';', header=None, encoding='latin1')

idx_inicio = df[df[0] == 'Entidad'].index[0]
idx_fin = df[df[0] == 'Gipuzkoa'].index[0]

df = df.loc[idx_inicio:idx_fin].reset_index(drop=True)

df = df.dropna(axis=1, how='all')

df = df.replace('"', '', regex=True)

df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

for col in df.columns[1:]:
    df[col] = df[col].str.replace(',', '.')
    df[col] = pd.to_numeric(df[col], errors='coerce')  

df = df.rename(columns={'Entidad': 'Territorio'})
df['Territorio'] = df['Territorio'].replace({'Araba/Álava': 'Araba'})

df.to_csv('./data/mujeres_electas_limpio.csv', index=False, sep=',')

df.head()


#Esto sigue siendo cada 1000 habitantes

,Territorio,2023,2019,2015,2011,2007,2003,1999,1995
0,Araba,44.78,42.89,35.73,31.09,26.70,22.76,22.00,17.85
1,Bizkaia,47.67,46.68,42.30,40.80,37.52,29.63,26.86,20.40
2,CAE,47.24,46.28,41.43,38.83,35.92,27.90,26.94,21.44
3,Gipuzkoa,47.77,47.26,42.87,39.79,38.04,27.95,29.20,24.36


In [13]:
import pandas as pd

df_poblacion = pd.read_csv('./data/poblacion_total_limpio.csv')
df_mujeres = pd.read_csv('./data/mujeres_electas_limpio.csv')

columnas_comunes = list(set(df_poblacion.columns[1:]) & set(df_mujeres.columns[1:]))
columnas_comunes = sorted(columnas_comunes, reverse=True)

df_poblacion = df_poblacion[['Territorio'] + columnas_comunes].sort_values('Territorio').reset_index(drop=True)
df_mujeres = df_mujeres[['Territorio'] + columnas_comunes].sort_values('Territorio').reset_index(drop=True)

df_totales = df_mujeres.copy()
for col in columnas_comunes:
    df_totales[col] = ((df_mujeres[col] / 1000) * df_poblacion[col]).round().astype('Int64')

df_totales.to_csv('./data/solicitantes_totales.csv', index=False)

print(df_totales.head())


  Territorio    2023    2019   2015   2011   2007   2003
0      Araba   14905   14067  11497   9909   8163   6624
1    Bizkaia   54535   53348  48283  47018  42696  33181
2        CAE  103774  101261  90036  84418  76545  58310
3   Gipuzkoa   34388   33895  30437  27968  26146  18980
